In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

cap = cv2.VideoCapture('./squirrel.mp4') #Replace the path with 0 to open the camera

maxCorners = 100

color = np.random.randint(0, 255, (maxCorners, 3))
feature_params = dict( maxCorners = maxCorners,
                       qualityLevel = 0.2,
                       minDistance = 7,
                       blockSize = 7
                     )


lk_params = dict( winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
  

ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
  
mask = np.zeros_like(old_frame)
while(1):
      
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
  
    good_new = p1[st==1]
    good_old = p0[st==1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        a = int(a)
        b = int(b)
        c = int(c)
        d = int(d)

        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
          
    img = cv2.add(frame, mask)
  
    cv2.imshow('frame', img)
      
    
    k = cv2.waitKey(25)
    if k == 27:
        break
  
    
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
  
cv2.destroyAllWindows()
cap.release()

حرکت آن ها را lucas kanade هرچند برای این بخش توضیحاتی خواسته نشده بود، اما تعداد 100 نقطه از نقاطی که مستعد دنبال کردن هستند را یافته ایم و مطابق الگوریتم  
درچند فریم پشت سر هم نشان داده ایم

In [2]:
cap = cv2.VideoCapture('./dog.mp4') #Replace the path with 0 to open the camera


ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)  

mask[..., 1] = 255

while(1):
      
    ret, frame = cap.read()
    cv2.imshow("Original Video", frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
      
    
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
      
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
      
    cv2.imshow("Dense Optical Flow", rgb)
    prev_gray = gray
    
    
    k = cv2.waitKey(25)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\padidar\AppData\Local\Temp\ipykernel_10492\37178838.py:21: RuntimeWarning: invalid value encountered in cast
  mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)


In [33]:
import os, os.path
import cv2
from sklearn.preprocessing import StandardScaler 

parent_path_surp = "../Lab1/Cohn-Kanade/surprise videos/"
parent_path_happ = "../Lab1/Cohn-Kanade/happy videos/"

videos_surp = os.listdir(parent_path_surp)
videos_happ = os.listdir(parent_path_happ)

CropAndResize = lambda img: img[55:670:2, 355:845:2]


Happy_Videos = []
Happy_labels = []

idxs = np.arange(len(videos_surp))
np.random.shuffle(idxs)
dirs = list(np.array(videos_surp)[idxs])

for idx, sub_dir in enumerate(dirs):
    directory = parent_path_surp + sub_dir + "/"
    images = os.listdir(directory)
    X = []
    for img_num, img in enumerate(images):
        pic = cv2.cvtColor(cv2.imread(directory + img), cv2.COLOR_BGR2GRAY)
        X.append(CropAndResize(pic))
    Happy_Videos.append(X)
    Happy_labels.append(0)


Surprise_Videos = []
Surprise_labels = []

idxs = np.arange(len(videos_happ))
np.random.shuffle(idxs)
dirs = list(np.array(videos_happ)[idxs])

for idx, sub_dir in enumerate(dirs):
    directory = parent_path_happ + sub_dir + "/"
    images = os.listdir(directory)
    X = []
    for img_num, img in enumerate(images):
        pic = cv2.cvtColor(cv2.imread(directory + img), cv2.COLOR_BGR2GRAY)
        X.append(CropAndResize(pic))
    Surprise_Videos.append(X)
    Surprise_labels.append(1)



تصاویر را خوانده، ماسک کرده و تغییر رزولوشن داده ایم.

In [65]:
lk_params = dict( winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

X = []
y = []

for hv in range(len(Happy_Videos)):
    prev_gray = Happy_Videos[hv][0]
    hist_bins = 8
    hist_angles_mean = np.zeros(hist_bins)
    hist_angles_max = np.zeros(hist_bins)
    hist_angles = np.zeros(hist_bins)
    
    i = 1
    while True:
        gray = Surprise_Videos[hv][i]
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        angle = np.arctan2(flow[..., 1], flow[..., 0])
        angle_deg = np.degrees(angle).flatten()
        hist, bins = np.histogram(angle_deg, bins=hist_bins, range=(-180, 180))

        hist_angles_mean += hist
        hist_angles_max = np.maximum(hist_angles_max, hist)

        prev_gray = gray.copy()

        i = i+1
        if i >= len(Surprise_Videos[hv]):
            break


    hist_angles_mean /= i

    whole_hist = np.concatenate([hist_angles_mean, hist_angles_max])
    

    X.append(whole_hist)
    y.append(0)





for sv in range(len(Surprise_Videos)):
    prev_gray = Surprise_Videos[sv][0]
    hist_bins = 8
    hist_angles_mean = np.zeros(hist_bins)
    hist_angles_max = np.zeros(hist_bins)
    hist_angles = np.zeros(hist_bins)
    
    i = 1
    while True:
        gray = Surprise_Videos[sv][i]
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        angle = np.arctan2(flow[..., 1], flow[..., 0])
        angle_deg = np.degrees(angle).flatten()
        hist, bins = np.histogram(angle_deg, bins=hist_bins, range=(-180, 180))

        hist_angles_mean += hist
        hist_angles_max = np.maximum(hist_angles_max, hist)
        
        prev_gray = gray.copy()

        i = i+1
        if i >= len(Surprise_Videos[sv]):
            break


    hist_angles_mean /= i

    whole_hist = np.concatenate([hist_angles_mean, hist_angles_max])
    

    X.append(whole_hist)
    y.append(1)






In [69]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
svm_classifier = SVC(kernel='linear', C=0.1, gamma=0.01)
svm_classifier.fit(X_train, y_train)
y_pred = 1^svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 66.67%
